### 다수의 웹페이지에서 정보 가져오기

여기까지는 가게 이름, 가게의 샌드위치 이름, 정보들이 더 담겨있는 url 주소까지 모두 확인해보았습니다.

* [Old Oak Tap BLT](https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/)

이 페이지를 살펴보면, 아래에 '가격'과 '주소'가 적혀져 있습니다. 그러니 여기에서 각각 가격과 주소를 얻어보도록 하겠습니다. 또 개발자 도구를 사용해 이 정보가 들어있는 태그를 확인해봅시다. 보니 p 태그에 addy class네요.

우선 그 전에 우리가 저장한 csv 파일을 불러와보겠습니다. 필요한 모듈들도 함께 뽑아와 보구요.

In [ ]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen 
import pandas as pd

df = pd.read_csv('03. best_sandwiches_list_chicago.csv', encoding='utf-8', index_col=0)
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


첫번째 사이트부터 들어가보면, 아까 들어있는 태그가 p 태그에 addy class죠?

In [ ]:
df['URL'][0]

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0'}
res = urllib.request.Request(df['URL'][0], headers =headers)
html = urlopen(res).read()
soup = BeautifulSoup(html, "html.parser")

# soup

여기에서 가격과 주소가 들어있는 곳만 뽑아와 보면!

In [ ]:
soup.find('p', class_='addy')

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>

In [ ]:
price_tmp = soup.find('p', class_='addy').get_text()
price_tmp

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

이 정보를 또 쪼개봐야지요.

In [ ]:
price_tmp.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

In [ ]:
price_tmp.split()[0]

'$10.'

가격에 .이 붙어있으니까 점을 없애보죠.

In [ ]:
price_tmp.split()[0][:-1]

'$10'

그럼 이제는 주소를 한 번 봅시다. 이것을 보면, 항상 뒤에서 두개를 제외하고까지가 주소인 것을 알 수 있습니다. (다른 주소들도 확인해보세요.)

In [ ]:
price_tmp.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

In [ ]:
# 주소만 가져오기
price_tmp.split()[1:-2]

['2109', 'W.', 'Chicago', 'Ave.,']

주소만 가져와봤는데, 이것이 다 따로따로 떨여져 있죠? 이것을 하나로 합하려면 join 함수를 사용하면 됩니다.

* [파이썬 join 함수](https://wikidocs.net/13#join)


In [ ]:
' '.join(price_tmp.split()[1:-2])

'2109 W. Chicago Ave.,'

그럼 이제 주소는 해결이 됐고, 가격과 주소가 잘 뽑아지는지 3개만 뽑아서 확인을 해보겠습니다.


In [ ]:
price = []
address =[]

for n in df.index[:3]:
    res = urllib.request.Request(df['URL'][n], headers =headers)
    html = urlopen(res).read()
    soup_tmp = BeautifulSoup(html, "html.parser")

    gettings = soup_tmp.find('p', 'addy').get_text()
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

In [ ]:
price

['$10', '$9', '$9.50']

In [ ]:
address

['2109 W. Chicago Ave.,', '800 W. Randolph St.,', '445 N. Clark St.,']

잘 나오네요!

### tqdm 모듈을 사용해 50 페이지에 접근하기

tqdm은 웹 크롤링할 때 상태 진행이 어떻게 되는지 알려주는 바형태의 시각화입니다. 앞서 뽑아낼 정보들이 50페이지를 모두 확인해보면서 추출하는 것이기 때문에 이렇게 얼마나 진행되었는지 알려주는 것이 있으면, 더 편하게 정보를 얻을 수 있겠죠.

* terminal에서 conda install -c conda-forge tqdm 해주기


아래와 같이 tqdm을 import 해주고, 앞서 한 대로 50 페이지를 모두 들어가 가격과 주소를 가져와 보도록 하죠. (코드를 그냥 쓰기 전에 먼저 코드를 어떻게 쓸 지 생각해보고 보는 것이 좋습니다)

In [ ]:
from tqdm import tqdm_notebook

price = []
address = []

for n in tqdm_notebook(df.index): 
    res = urllib.request.Request(df['URL'][n], headers =headers)
    html = urlopen(res).read()
    soup_tmp = BeautifulSoup(html, "html.parser")

    gettings = soup_tmp.find('p', 'addy').get_text()
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))


<ipython-input-42-1cc37d5c9b5c>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(df.index):


  0%|          | 0/50 [00:00<?, ?it/s]

정말 오래 걸리죠? 제대로 가져와봤는지 확인해봅시다. (제대로 안 됐으면 완전 짱나겠죠ㅎㅎ)

In [ ]:
price[:10]

['$10', '$9', '$9.50', '$9.40', '$10', '$7.25', '$16', '$10', '$9', '$17']

In [ ]:
address[:30]

['2109 W. Chicago Ave.,',
 '800 W. Randolph St.,',
 '445 N. Clark St.,',
 '914 Noyes St., Evanston,',
 '825 W. Fulton Mkt.,',
 '100 E. Walton',
 '1639 S. Wabash Ave.,',
 '2211 W. North Ave.,',
 '3619 W. North Ave.,',
 '3267 S. Halsted St.,',
 '2537 N. Kedzie Blvd.,',
 'Multiple',
 '3124 N. Broadway,',
 '3455 N. Southport Ave.,',
 '2657 N. Kedzie Ave.,',
 '1120 W. Grand Ave.,',
 '1141 S. Jefferson St.,',
 '333 E. Benton Pl.,',
 '1411 N. Wells St.,',
 '1747 N. Damen Ave.,',
 '3209 W. Irving Park',
 'Multiple',
 '5347 N. Clark St.,',
 '2954 W. Irving Park Rd.,',
 'Multiple',
 '191 Skokie Valley Rd., Highland Park,',
 'Multiple',
 '1818 W. Wilson Ave.,',
 '2517 W. Division St.,',
 '218 W. Kinzie']

어, 그런데 여기 multiple이란게 있죠. 샌드위치 가게가 체인점이라 여러군데에 있나봐요. 일단 이것을 알아두시고 어떻게 처리할 지 고민을 해봅시다.

In [ ]:
len(price), len(address)

(50, 50)

와핫, 다 잘 가져온 것 같네요. (여러분들은 price와 address를 모두 확인해보세요~) 이 정보들을 데이터프레임에 새로운 컬럼에 넣어볼게요.

In [ ]:
df['Price'] = price 
df['Address'] = address

df.head()

,Rank,Cafe,Menu,URL,Price,Address
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


유후, 잘 됐구요! 깔끔하게 Rank를 index로 넣어봅시다.

In [ ]:
df.set_index('Rank', inplace=True)
df.head()

,Cafe,Menu,URL,Price,Address
Rank,,,,,
1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


그러면 이 크롤링한 정보들을 잘 예쁘게 저장!합시다. 
다 해놓고 저장을 까먹으면 안 되겠죠?

In [ ]:
df.to_csv('03. best_sandwitches_list_chicago2.csv', sep=',', encoding='utf-8')